# Scanpy anndata conversion to Seurat object mediated by loom

This notebook describes a method to convert anndata files (native to Scanpy) to Seurat objects. SeuratDisk has a function for this very purpose, but at this time (last checked: 14/05/2021) it is not functional, not even by following the provided vignette. 

In this alternative method, the anndata object is converted to loom and then converted to Seurat. The loom file can be edited to ensure that it is correctly loaded onto Seurat.

### Loading required packages

In [ ]:
import anndata
import numpy as np
import loompy

In [ ]:
%%R
library(Seurat)
library(SeuratDisk)

### Variable inputs

In [ ]:
# Variables for reading in data:
anndata_file = 'outputs/filename.h5ad' # path for the input anndata object
loom_file = anndata_file + '.loom'
seurat_file = anndata_file + '.rds'

# outputs will be stored in the same folder

## 1. Export (with Scanpy)

In [ ]:
adata = anndata.read(anndata_file)
adata.write_loom(loom_file)

In [ ]:
adata

## 2. Editing (with Loompy)(optional)

In [ ]:
loomfile = loompy.connect(loom_file)

In [ ]:
# The row features
loomfile.ra.keys()

In [ ]:
# The column features
loomfile.ca.keys()

In [ ]:
# Gene symbols are generally stored in 'var_names' or 'Genes' in the row features. The gene_ids row often contains the ENSEMBL ID.
loomfile.ra.gene_ids

In [ ]:
## If you wish to change the name of a feature, you can do so by:
#loomfile.ra.Gene = loomfile.ra.gene_ids
#del loomfile.ra.var_names

## 3. Import (with Seurat)

In [ ]:
%%R
newdata <- Connect(filename = loom_file, mode = "r")
newdata

In [ ]:
%%R
# Remember to set correctly the cells and features parameters. Documentation: https://www.rdocumentation.org/packages/Seurat/versions/3.1.0/topics/as.Seurat
seuratfile <- as.Seurat(newdata, cells = "obs_names", features = "var_names", assay = "RNA", verbose = TRUE)

In [ ]:
%%R
saveRDS(seuratfile, file = seurat_file)